In [1]:
import os

In [2]:
%pwd

'x:\\deep_learning\\text-summarization\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'x:\\deep_learning\\text-summarization'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config= self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config =DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [15]:
import os
import urllib.request as request
import zipfile
from text_summarizer.logging import logger
from text_summarizer.utils.common import get_size


In [16]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig) :
        self.config=config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers= request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )

            logger.info(f"{filename} download with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size : {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)


In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-02 08:53:16,239: INFO :common :yaml file: config\config.yaml loaded successfully]
[2024-08-02 08:53:16,239: INFO :common :yaml file: params.yaml loaded successfully]
[2024-08-02 08:53:16,239: INFO :common :created directory at: artifacts]
[2024-08-02 08:53:16,254: INFO :common :created directory at: artifacts/data_ingestion]
[2024-08-02 08:53:18,877: INFO :2700526799 :artifacts/data_ingestion/corpus.zip download with following info: 
Connection: close
Content-Length: 3948308
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "36dcb2e3ae577c8bdd925f521a82a07cef22432064a6f0a4c90a28961be82472"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9508:37BC79:539DE1:612CE5:66AC5126
Accept-Ranges: bytes
Date: Fri, 02 Aug 2024 03:23:19 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4727-BOM
X-Cache: M